In [12]:
import numpy as np
import pandas as pd

import joblib

## SHapley Additive exPlanations.
import shap

## Own specific functions 
from functions import *

In [6]:
df_clients_to_predict = pd.read_csv(r"datasets\finals_datasets\df_clients_to_predict_feature_selected.csv")

In [8]:
df_analysis(df_clients_to_predict, "df_clients_to_predict", analysis_type="header")


Analysis Header of df_clients_to_predict dataset
--------------------------------------------------------------------------------
- Dataset shape:			 48744 rows and 150 columns
- Total of NaN values:			 0
- Percentage of NaN:			 0.0 %
- Total of infinite values:		 0
- Percentage of infinite values:	 0.0 %
- Total of full duplicates rows:	 0
- Total of empty rows:			 0
- Total of empty columns:		 0
- Unique indexes:			 True
- Memory usage:				 55.8 MB


In [9]:
df_clients_to_predict = memory_optimization(df_clients_to_predict)

In [10]:
df_analysis(df_clients_to_predict, "df_clients_to_predict", analysis_type="header")


Analysis Header of df_clients_to_predict dataset
--------------------------------------------------------------------------------
- Dataset shape:			 48744 rows and 150 columns
- Total of NaN values:			 0
- Percentage of NaN:			 0.0 %
- Total of infinite values:		 0
- Percentage of infinite values:	 0.0 %
- Total of full duplicates rows:	 0
- Total of empty rows:			 0
- Total of empty columns:		 0
- Unique indexes:			 True
- Memory usage:				 24.1 MB


In [11]:
model = joblib.load("models\model_20220217.pkl")

In [13]:
explainer = shap.TreeExplainer(model)

In [14]:
NOT_TRAINING_COLUMNS  = ["TARGET", "SK_ID_CURR", "SK_ID_BUREAU", "SK_ID_PREV", "index"] 
TRAINING_COLUMNS = [col for col in df_current_clients.columns if col not in NOT_TRAINING_COLUMNS]

In [15]:
df_clients_to_predict = df_clients_to_predict[TRAINING_COLUMNS]

In [ ]:
shap_values = explainer.shap_values(df_clients_to_predict)